**One-Step Binomial Model**


In [37]:
import math

def one_step_call_price(S0, E, u, d, r, T):
    """Price a European call option using the one-step binomial model."""
    # Risk-neutral probability
    p = (math.exp(r * T) - d) / (u - d)
    
    # Payoffs at maturity
    Cu = max(S0 * u - E, 0)
    Cd = max(S0 * d - E, 0)
    
    # Discounted expected value
    C0 = math.exp(-r * T) * (p * Cu + (1 - p) * Cd)
    
    return {
        "Cu": round(Cu,3), "Cd": round(Cd,3),
        "p": round(p,5), "C0": round(C0,3)
    }

In [38]:
# Example 8.5 from the text
S0, E, u, d, r, T = 40, 42, 1.1, 0.9, 0.12, 0.25
result = one_step_call_price(S0, E, u, d, r, T)
print(result)

#Expected output: {'Cu': 2, 'Cd': 0, 'p': 0.65227, 'C0': 1.266}

{'Cu': 2.0, 'Cd': 0, 'p': 0.65227, 'C0': 1.266}


**Two-Step Binomial Model**

In [39]:
def two_step_option_price(S0, E, u, d, r, T, option_type="call"):
    """Price a European option using a two-step binomial model."""
    t = T / 2
    p = (math.exp(r * t) - d) / (u - d)
    
    # Payoffs at maturity
    Suu = S0 * (u ** 2)
    Sud = S0 * u * d
    Sdd = S0 * (d ** 2)
    
    if option_type == "call":
        Cuu = max(Suu - E, 0)
        Cud = max(Sud - E, 0)
        Cdd = max(Sdd - E, 0)
    else:  # put option
        Cuu = max(E - Suu, 0)
        Cud = max(E - Sud, 0)
        Cdd = max(E - Sdd, 0)
    
    # Price at t=0
    C0 = math.exp(-r * T) * (
        (p ** 2) * Cuu +
        2 * p * (1 - p) * Cud +
        ((1 - p) ** 2) * Cdd
    )
    
    return {
        "p": round(p,5),
        "Cuu": round(Cuu,3), "Cud": round(Cud,3), "Cdd": round(Cdd,3),
        "C0": round(C0,3)
    }

In [40]:
# Example 9.6: Two-step put option
S0, E, u, d, r, T = 40, 32, 1.2, 0.8, 0.1, 0.5
result = two_step_option_price(S0, E, u, d, r, T, option_type="put")
print(result)

# Expected output: {'p': 0.56329, 'Cuu': 0, 'Cud': 0, 'Cdd': 6.4, 'C0': 1.161}

{'p': 0.56329, 'Cuu': 0, 'Cud': 0, 'Cdd': 6.4, 'C0': 1.161}


**Multi-Step (N-Step) Binomial Model**

In [41]:
def binomial_option_price(S0, E, r, T, u, d, N, option_type="call"):
    """Price a European option using an N-step binomial tree."""
    dt = T / N
    p = (math.exp(r * dt) - d) / (u - d)
    
    # Prices at maturity
    prices = [S0 * (u ** j) * (d ** (N - j)) for j in range(N + 1)]
    
    # Option payoffs at maturity
    if option_type == "call":
        values = [max(price - E, 0) for price in prices]
    else:
        values = [max(E - price, 0) for price in prices]
    
    # Work backwards through the tree
    for i in range(N - 1, -1, -1):
        values = [
            math.exp(-r * dt) * (p * values[j + 1] + (1 - p) * values[j])
            for j in range(i + 1)
        ]
    
    return {
        "p": round(p,5),
        "option_price": round(values[0],2)
    }

In [42]:
# Example: Multi-step call option (use same parameters as Example 8.5 but N=3)
S0, E, r, T, u, d, N = 40, 42, 0.12, 0.25, 1.1, 0.9, 3
print(binomial_option_price(S0, E, r, T, u, d, N, "call"))

{'p': 0.55025, 'option_price': 2.44}
